In [1]:
#|default_exp utils

# Utils

In [2]:
#|export
import os
import re
import numpy as np
from fastcore.all import *

In [3]:
#|export
def convert_uuids_to_indices():
    cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES", "")
    uuids = re.findall(r"\b[0-9a-fA-F]{8}(?:-[0-9a-fA-F]{4}){3}-[0-9a-fA-F]{12}\b", cuda_visible_devices)

    if uuids:
        indices = [str(i) for i in range(len(uuids))]
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(indices)

In [4]:
#|export
def percent_rel_error_(N, N_mc, N_mc_t0):
    """
    Return the percentage relative error with respect to the MC simulation
    See Eq. 23 of the paper by Giudici et al. (2023), 'Space debris density 
    propagation through a vinite volume method'
    Input:
        N: Predicted number of in-orbit elements over time
        N_mc: Number of in-orbit elements from the MC simulation over time
        N_mc_t0: Number of in-orbit elements from the MC simulation at t0
    Output:
        The percentage relative error (float)
    """
    return 100 * np.abs(N_mc - N) / N_mc_t0

In [5]:
# Test
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([1, 2, 3, 4, 5])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.zeros_like(res))

# Test 2
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([2, 4, 6, 8, 10])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.array([ 50., 100., 150., 200., 250.]))



In [6]:
#| export
def calculate_sample_idxs(simulation_idxs, samples_per_sim):
    indices = []
    for sim in simulation_idxs:
        start_idx = sim * samples_per_sim
        end_idx = start_idx + samples_per_sim
        indices.extend(range(start_idx, end_idx))
    return indices

In [11]:
# Test
sim_data = np.random.rand(10, 4, 36, 99) # sims x samples x h x w
simulation_idxs = [1, 3]
samples_per_sim = sim_data.shape[1]
sample_idxs = calculate_sample_idxs(simulation_idxs, samples_per_sim)
sample_idxs

[4, 5, 6, 7, 12, 13, 14, 15]